For the GCN, we need to create an adjacency matrix of the graph. As the size of the graph is massive (1.2 million paper nodes), we need to use a sparse matrix due to its efficiency. It is also a requirement for training the model.

First, let's practice with just the citation graph.


In [38]:
import numpy as np
from scipy.sparse import csr_matrix
import pickle
import os

In [39]:
#authors = np.load("test_data/author_array_test.npy")
path = os.getcwd()
size = "small"
edgelist = np.load("{}/test_data_collection/test_data_{}/cites_{}.npy".format(path,size,size))
labels = np.load("{}/test_data_collection/test_data_{}/labels_{}.npy".format(path,size,size))
papers = np.load("{}/test_data_collection/test_data_{}/papers_{}.npy".format(path,size,size))
#author_to_paper = np.load("test_data/edgelist_writes_test.npy")

# Use this to fix the difference
#np.setdiff1d(papers, np.unique(paper_to_paper))


ind.dataset_str.y => the one-hot labels of the labeled training instances as numpy.ndarray object;
    
ind.dataset_str.ty => the one-hot labels of the test instances as numpy.ndarray object;

ind.dataset_str.ally => the labels for instances in ind.dataset_str.allx as numpy.ndarray object;

ind.dataset_str.test.index => the indices of test instances in graph, for the inductive setting as list object.

In [40]:
edgelist_reindexed = []
papers_reindexed = []
mapping = {}

# reindex edges and papers to start from 0
for k in range(len(papers)):
    mapping[papers[k]] = k
    papers_reindexed.append(k)
papers = np.array(papers_reindexed)

for m in range(len(edgelist.T)):
    edge = edgelist[:,m]
    edgelist_reindexed.append([mapping[edge[0]],mapping[edge[1]]])
edgelist = np.array(edgelist_reindexed)


In [41]:
labels = np.load("{}/test_data_collection/test_data_{}/labels_{}.npy".format(path,size,size)) # 153 classes in the data
known_labels = np.where(labels!=-1)[0]
train_test_index_split= int(0.8*len(known_labels))
train_index = []
test_index = []
j = 0
label_matrix = np.zeros((len(labels),153))
for i in range(len(labels)):
    if not labels[i] == -1:
        label_matrix[i,int(labels[i])] = 1
        if j < train_test_index_split:
            train_index.append(i)
            j += 1
        else:
            test_index.append(i)

train_labels = label_matrix[train_index,:]
with open('/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.y', 'wb') as handle:
    pickle.dump(train_labels, handle, protocol=pickle.HIGHEST_PROTOCOL)

test_labels = label_matrix[test_index,:]
with open('/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.ty', 'wb') as handle:
    pickle.dump(test_labels, handle, protocol=pickle.HIGHEST_PROTOCOL)

label_matrix_no_test = np.delete(label_matrix, test_index, axis=0) # remove test index
with open('/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.ally', 'wb') as handle:
    pickle.dump(label_matrix_no_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.test.index', 'w') as f:
    for index in test_index:
        f.write(str(index) + "\n")


train 0
train 4
train 9
train 17
train 23
train 30
train 37
train 40
train 67
train 69
train 71
train 78
train 99
train 101
train 104
train 110
train 122
train 123
train 126
train 128
train 130
train 134
train 138
train 159
train 169
train 172
train 180
train 183
train 187
train 193
train 198
train 227
train 261
train 279
train 282
train 296
train 297
train 298
train 301
train 311
train 313
train 315
train 322
train 359
train 362
train 369
train 378
train 381
train 382
train 383
train 386
train 402
train 404
train 405
train 411
train 417
train 418
train 422
train 431
train 435
train 444
train 450
train 455
train 457
train 458
train 468
train 474
train 475
train 478
train 481
train 488
train 500
train 501
train 505
train 509
train 512
train 519
train 527
train 534
train 535
train 540
train 554
train 558
train 575
train 583
train 599
train 603
train 607
train 611
train 612
train 613
train 629
train 633
train 634
train 640
train 652
train 654
train 655
train 663
train 665
train 682
train 

ind.dataset_str.x => the feature vectors of the training instances as scipy.sparse.csr.csr_matrix object;

ind.dataset_str.tx => the feature vectors of the test instances as scipy.sparse.csr.csr_matrix object;

ind.dataset_str.allx => the feature vectors of both labeled and unlabeled training instances
        (a superset of ind.dataset_str.x) as scipy.sparse.csr.csr_matrix object;

In [42]:
feature_matrix = np.load("{}/test_data_collection/test_data_{}/features_{}.npy".format(path,size,size))
feature_matrix_no_test = np.delete(feature_matrix, test_index, axis=0) # remove test index
feature_matrix_train = feature_matrix[train_index]
feature_matrix_test = feature_matrix[test_index]

sparse_feature_matrix = csr_matrix(feature_matrix_no_test)
with open('/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.allx', 'wb') as handle:
    pickle.dump(sparse_feature_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

sparse_feature_matrix_train = csr_matrix(feature_matrix_train)
with open('/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.x', 'wb') as handle:
    pickle.dump(sparse_feature_matrix_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

sparse_feature_matrix_test = csr_matrix(feature_matrix_test)
with open('/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.tx', 'wb') as handle:
    pickle.dump(sparse_feature_matrix_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

ind.dataset_str.graph => a dict in the format {index: [index_of_neighbor_nodes]} as collections.defaultdict object;

In [43]:
G = dict()
for edge in edgelist:
    if edge[0] in G:
        G[edge[0]].append(int(edge[1]))
    else:
        G[edge[0]] = [int(edge[1])]

    if edge[1] in G: 
        G[edge[1]].append(int(edge[0]))
    else:
        G[edge[1]] = [int(edge[0])]

for no_edge in np.setdiff1d(papers_reindexed, np.unique(edgelist)):
    G[no_edge] = [int(no_edge)]

with open('/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.graph', 'wb') as handle:
    pickle.dump(G, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [44]:
import pandas as pd

allx = pd.read_pickle(r'/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.allx')
ally = pd.read_pickle(r'/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.ally')
graph = pd.read_pickle(r'/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.graph')
index = pd.read_csv(r'/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.test.index')
tx = pd.read_pickle(r'/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.tx')
ty = pd.read_pickle(r'/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.ty')
x = pd.read_pickle(r'/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.x')
y = pd.read_pickle(r'/Users/Samu/Desktop/Koulu/Gradu/gcn/gcn/data/ind.mag_prac.y')

print(allx.shape)
print(ally.shape)
print(len(graph))
print(len(index))
print(tx.shape)
print(ty.shape)
print(x.shape)
print(y.shape)

(1535, 768)
(1535, 153)
1582
46
(47, 768)
(47, 153)
(186, 768)
(186, 153)
